In [1]:
import pandas as pd
import numpy as np

In [24]:
def extract_header(filename, comment_prefix, header_row_index, header_row_delimiter):
    with open(filename, 'r') as file:
        lines = file.readlines()

        header = []
        for line in lines:
            if line[0] == comment_prefix:
                header.append(line)

    
    header_extract = header[header_row_index]
    header_extract = header_extract.replace(comment_prefix, '')
    split_header = header_extract.split(header_row_delimiter)
    return_header = [x.strip() for x in split_header]
    
    return return_header


In [32]:
header = extract_header('Land_and_Ocean_summary.txt', 
                         comment_prefix='%', 
                         header_row_index=-1, 
                         header_row_delimiter=',')



['Year', 'Annual Anomaly', 'Annual Unc.', 'Five-year Anomaly', 'Five-year Unc.', 'Annual Anomaly', 'Annual Unc.', 'Five-year Anomaly', 'Five-year Unc.']


In [38]:
def read_anomaly_data_into_dataframe(filename, header, max_cols, comment_prefix):
    header = header[:max_cols]
    anomaly_df = pd.read_table(filename, 
                               delimiter=r'\s+', 
                               comment=comment_prefix, 
                               names=header, 
                               usecols=range(max_cols))
    return anomaly_df

In [40]:
anomaly_df = read_anomaly_data_into_dataframe('Land_and_Ocean_summary.txt', header, max_cols=5, comment_prefix='%')

,Year,Annual Anomaly,Annual Unc.,Five-year Anomaly,Five-year Unc.
0,1850,-0.532,0.154,NaN,NaN
1,1851,-0.405,0.169,NaN,NaN
2,1852,-0.395,0.165,-0.423,0.121
3,1853,-0.418,0.158,-0.384,0.109
4,1854,-0.363,0.142,-0.400,0.090
...,...,...,...,...,...
164,2014,0.675,0.045,0.728,0.040
165,2015,0.814,0.043,0.777,0.041
166,2016,0.949,0.044,0.808,0.040
167,2017,0.834,0.045,NaN,NaN


In [ ]:
def anomaly_avg_per_dec(anomaly_df):
    
